In [5]:
import pandas as pd
import numpy as np
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

pga_clean = pd.read_csv('pgatour_22_cleaned.csv',index_col=0)
pga_clean.head()

,player_id,player,tournament_id,tournament,course,city,state,purse,no_cut,finish,position,made_cut,rounds_played,strokes,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
0,9261,Abraham Ancer,401353224,The Memorial Tournament pres. by Nationwide,Muirfield Village Golf Club,Dublin,OH,12.0,False,T32,32.0,True,4,289,0.20,-0.13,-0.08,0.86,0.65,0.85
1,5548,Adam Hadwin,401353224,The Memorial Tournament pres. by Nationwide,Muirfield Village Golf Club,Dublin,OH,12.0,False,T18,18.0,True,4,286,0.36,0.75,0.31,0.18,1.24,1.60
2,4989,Anirban Lahiri,401353224,The Memorial Tournament pres. by Nationwide,Muirfield Village Golf Club,Dublin,OH,12.0,False,CUT,0.0,False,2,147,-0.56,0.74,-1.09,0.37,0.02,-0.54
3,6015,Adam Long,401353224,The Memorial Tournament pres. by Nationwide,Muirfield Village Golf Club,Dublin,OH,12.0,False,CUT,0.0,False,2,151,-1.46,-1.86,-0.02,0.80,-1.08,-2.54
4,3832,Alexander Noren,401353224,The Memorial Tournament pres. by Nationwide,Muirfield Village Golf Club,Dublin,OH,12.0,False,CUT,0.0,False,2,148,0.53,-0.36,-1.39,0.19,-1.56,-1.04
